In [1]:
import numpy as np 
import pandas as pd 
import string
from string import digits 
import re 
import math 
import contractions 
import gc

In [2]:
# What version of Python do you have?
import sys 
import platform
print(f"python Platform: {platform.platform()}")
print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")

python Platform: macOS-15.4-arm64-arm-64bit
Python: 3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]
Pandas: 2.2.3


## Defining Token

In [3]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'
max_sequence_length = 90 #I've checked it in exploratary analysis-> 150 is ideal
min_sequence_length = 2 # will remove all sentence with only single word

## Preparing Data

### Read hindi file clean a bit and write it down again.

In [4]:
base_path = '/Users/siddharthchaudhary/Documents/git_repos/AI-Models/Transformer_from_scratch/Dataset/'

In [5]:
hindi_english_data_path =  base_path + 'Dataset_English_Hindi.csv'
pre_process_en_file = base_path + 'vocab_and_training_files/pre_process_en.txt'
pre_process_hn_file = base_path + 'vocab_and_training_files/pre_process_hn.txt'
train_en_file = base_path + 'vocab_and_training_files/train_en.txt'
train_hn_file = base_path + 'vocab_and_training_files/train_hn.txt'
english_vocab_file = base_path + 'vocab_and_training_files/english_vocab.txt'
hindi_vocab_file = base_path + 'vocab_and_training_files/hindi_vocab.txt'

In [6]:
lines = (pd. read_csv(hindi_english_data_path).rename (columns = {'English': 'english', 'Hindi': 'hindi'}))
# lines = lines [:1000000]
# lines = lines[:20000]
lines.head()

,english,hindi
0,Help!,बचाओ!
1,Jump.,उछलो.
2,Jump.,कूदो.
3,Jump.,छलांग.
4,Hello!,नमस्ते।


In [7]:
lines.tail()

,english,hindi
130471,Examples of art deco construction can be found...,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
130472,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
130473,"As for the other derivatives of sulphur , the ...","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द..."
130474,its complicated functioning is defined thus in...,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .
130475,They've just won four government contracts to ...,हाल ही में उन्हें सरकारी ठेका मिला है करीब सौ ...


In [8]:
missing_data = lines.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values"})
missing_data["% of Missing Values"] = round((missing_data["Total No. of Missing Values"]/len(lines))*100,2)

In [9]:
round(lines.describe().T,2)

,count,unique,top,freq
english,130474,126959,(Laughter),555
hindi,130164,100228,(हँसी),212


In [10]:
lines.drop_duplicates(inplace = True)

In [11]:
lines = lines.dropna(subset=["hindi", "english"])
lines.head()

,english,hindi
0,Help!,बचाओ!
1,Jump.,उछलो.
2,Jump.,कूदो.
3,Jump.,छलांग.
4,Hello!,नमस्ते।


In [12]:
lines.shape

(127375, 2)

In [13]:
# Lowercase all characters
lines['english'] = lines['english'].str.lower()

In [14]:
# Remove HTML
def remove_html_tags(text):
    # Corrected pattern: Only allows alphanumeric characters and spaces
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, ' ', text)
    return text


lines['english'] = lines['english'].apply(remove_html_tags)

In [15]:
# remove URLS
def remove_url(text):
    # Correct regex to match URLs
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(' ', text)
    
lines['english'] = lines['english'].apply(remove_url)
lines['hindi'] = lines['hindi'].apply(remove_url)
lines.head()

,english,hindi
0,help,बचाओ!
1,jump,उछलो.
2,jump,कूदो.
3,jump,छलांग.
4,hello,नमस्ते।


In [16]:
# Contraction:- y're → you are 
def expand_contractions (text):
    expanded_text = contractions.fix(text)
    return expanded_text
    
lines['english'] = lines['english'].apply(expand_contractions)
lines.head()

,english,hindi
0,help,बचाओ!
1,jump,उछलो.
2,jump,कूदो.
3,jump,छलांग.
4,hello,नमस्ते।


In [17]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "А3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAO": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "NRN": "No Reply Necessary",
    "OIC":"Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN" : "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WUF": "Where Are You From?",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "LOL": "Laughing out loud",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "IDC": "I don't care",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "LMAO": "Laughing my a** off",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [18]:
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

lines['english'] = lines['english'].apply(chat_conversion)

In [19]:
lines.head()

,english,hindi
0,help,बचाओ!
1,jump,उछलो.
2,jump,कूदो.
3,jump,छलांग.
4,hello,नमस्ते।


In [20]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" 
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF" 
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002702-\U000027B0" 
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

In [21]:
lines.head()

,english,hindi
0,help,बचाओ!
1,jump,उछलो.
2,jump,कूदो.
3,jump,छलांग.
4,hello,नमस्ते।


In [22]:
lines.shape

(127375, 2)

In [23]:
lines['english'] = lines['english'].apply(remove_emoji)
lines['hindi'] = lines['hindi'].apply(remove_emoji)

In [25]:
# Function to preprocess text based on language and remove numbers
def preprocess_text(text, language='english'):
    if not isinstance(text, str):
        return text  # Return as-is if input is not a string

    if language == 'english':
        # Remove non-alphanumeric characters
        return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    elif language == 'hindi':
        # Remove characters outside the Hindi Unicode range and Hindi numbers
        return re.sub(r'[^\u0900-\u097F\s]', ' ', text)
    else:
        # Raise error for unsupported languages
        raise ValueError("Unsupported language. Supported languages are 'english' and 'hindi'.")

# Apply preprocessing to 'lines' DataFrame
lines['english'] = lines['english'].apply(lambda x: preprocess_text(x, language='english'))
lines['hindi'] = lines['hindi'].apply(lambda x: preprocess_text(x, language='hindi'))


In [26]:
# Get the number of words count in each sentence for hindi and english
lines['length_eng']=lines['english'].apply(lambda x: len(x.split(" ")) )
lines['length_hin']=lines['hindi'].apply(lambda x: len(x.split(" ")))

In [27]:
# Drop rows where the "hindi" column contains English words OR the "english" column contains Hindi words
def contains_hindi(text):
    # Check for any character in the Unicode range for Devanagari script
    return bool(re.search(r'[\u0900-\u097F]', text))

# Define a function to check for English words in a Hindi sentence
def contains_english(text):
    # Check for any Latin script character
    return bool(re.search(r'[a-zA-Z]', text))


lines = lines[~((lines['hindi'].apply(contains_english)) | (lines['english'].apply(contains_hindi)))]

# Display the cleaned DataFrame
lines.head()

,english,hindi,length_eng,length_hin
0,help,बचाओ,1,2
1,jump,उछलो,1,2
2,jump,कूदो,1,2
3,jump,छलांग,1,2
4,hello,नमस्ते।,1,1


In [28]:
# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip ())

In [29]:
# Define a function to check for English alphabets in a Hindi sentence
def contains_english_alphabets(text):
    # Check for any Latin script characters (English alphabets)
    return bool(re.search(r'[a-zA-Z]', text))

# Identify rows where Hindi sentences contain English alphabets
lines['contains_english'] = lines['hindi'].apply(contains_english_alphabets)

# Filter out sentences containing English alphabets
filtered_lines = lines[lines['contains_english']]

# Display the result
filtered_lines.head()

Empty DataFrame
Columns: [english, hindi, length_eng, length_hin, contains_english]
Index: []


In [30]:
lines[lines['length_eng'] > max_sequence_length].shape

(462, 5)

In [31]:
lines[lines['length_hin'] > max_sequence_length].shape

(1584, 5)

In [32]:
lines = lines[lines['length_eng']<=max_sequence_length]
lines = lines[lines['length_hin']<=max_sequence_length]

In [33]:
lines = lines[lines['length_eng']>=min_sequence_length]
lines = lines[lines['length_hin']>=min_sequence_length]

In [34]:
lines.shape

(124084, 5)

In [35]:
print("maximum length of Hindi Sentence ", max(lines ['length_hin'])) 
print("maximum length of English Sentence ", max (lines ['length_eng']))

maximum length of Hindi Sentence  90
maximum length of English Sentence  90


In [36]:
lines.head()

,english,hindi,length_eng,length_hin,contains_english
8,got it,समझे कि नहीं,2,4,False
9,i m ok,मैं ठीक हूँ।,3,3,False
11,come in,अंदर आ जाओ।,2,3,False
12,get out,बाहर निकल जाओ,2,4,False
13,go away,चले जाओ,2,3,False


In [37]:
hindi_sentences = lines['hindi'].values.tolist()
english_sentences = lines['english'].values.tolist()

In [38]:
### Get English and Hindi Vocabulary
all_eng_words = set()
for eng in lines['english']:
    all_eng_words.update(eng.split())

all_hindi_words = set ()
for hin in lines['hindi']:
    all_hindi_words.update(hin.split())

In [39]:
hindi_vocab = list(all_hindi_words)
print(len(hindi_vocab))
english_vocab = list(all_eng_words)
print(len(english_vocab))

68142
59893


### Now Writing the hindi, english text and vocab created in above 3 cells

In [40]:
file = open(pre_process_en_file,'w')
for sentence in english_sentences:
    file.write(sentence+"\n")
file.close()

In [41]:
file = open(pre_process_hn_file,'w')
for sentence in hindi_sentences:
    file.write(sentence+"\n")
file.close()

In [42]:
file = open(english_vocab_file, 'w')
for word in english_vocab: 
    file.write(word+"\n")
file.close()

In [43]:
file = open(hindi_vocab_file, 'w')
for word in hindi_vocab:
    file.write (word+"\n")
file.close()

In [44]:
# Delete dataframes to release memory
del lines 
del all_eng_words 
del all_hindi_words 
del hindi_sentences 
del english_sentences 
del hindi_vocab 
del english_vocab
# Run garbage collector
gc.collect()

0

### Reading the hindi, english text and vocab

In [45]:
with open(pre_process_en_file, 'r') as file:
    english_sentences = file.readlines()
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
english_sentences[:10]

['got it',
 'i m ok',
 'come in',
 'get out',
 'go away',
 'have fun',
 'have fun',
 'have fun',
 'i forgot',
 'i forgot']

In [46]:
with open(pre_process_hn_file, 'r') as file:
    hindi_sentences = file.readlines()
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]
hindi_sentences[:10]

['समझे कि नहीं',
 'मैं ठीक हूँ।',
 'अंदर आ जाओ।',
 'बाहर निकल जाओ',
 'चले जाओ',
 'मज़े करना।',
 'मौज करना।',
 'मज़े करो।',
 'मैं भूल गया।',
 'मैं भूल गई।']

In [47]:
with open(hindi_vocab_file, 'r') as file:
    hindi_vocab = file.readlines ()
hindi_vocab = [word.rstrip('\n') for word in hindi_vocab]
print(len(hindi_vocab) )
print(hindi_vocab[:50])

68142
['दीऋए', 'कपाट', 'मेगस्थनीज', 'पोखरण', 'मुंजुल', 'पड़ाव', 'पादरी', 'रामदास', 'दबोचनेवाली', 'तत्वावधान', 'केपटाउन', 'अन्याय', 'राय', 'छ्हेस्हिरे', 'छंबोबद्ध', 'सेवाशर्तों', 'रूद्रँ', 'उद्यत', 'हैअलबत्ता', 'आंशका', 'लाया।', 'इंटरनेशल', 'बालासोर', 'महापुरूष', 'वेबखोज', 'फीजी', 'परंतुप्रत्येक', 'जन्मशती', 'हसपताल', 'गुडिया', 'मुबाहिस', 'भरव', 'रोटियां', 'टेक्नालाजी', 'त्रिविशंति', 'प्रकल्प', 'कोनीमारा', 'छलकता', 'शिलाव्यादिकारम्', 'अचकननूमा', 'मानीटर', 'शाक्य', 'वीसा', 'बीड़ी', 'ब्युरो', 'डिफेंसर', 'हॅ।', 'गैरमऋजूदगी', 'किचलू', 'व्हीलचेयर']


In [48]:
with open (english_vocab_file, 'r') as file:
    english_vocab= file.readlines()
english_vocab = [word.rstrip('\n') for word in english_vocab]
print(len(english_vocab))
print(english_vocab[:50])

59893
['comgress', 'yajnik', 'alwaye', 'shoestring', 'kariyan', 'countribution', 'dwadashiyamaasai', 'deals', 'gangga', 'denied', 'squadron', 'muslmans', 'vidur', 'youthful', 'epitaph', 'dravida', 'acheived', 'cambridge', 'counterterrorism', 'abodes', 'frictionlessly', 'cure', 'swarthmore', 'changesin', 'impasse', 'shampens', 'yoddh', 'palpable', 'somashekar', 'skull', 'incarcerate', 'uprooted', 'crux', 'aangira', 'sundarvan', 'doostana', 'sniffable', 'speaks', 'earthern', 'pawn', '1583', 'daryao', 'chambered', 'testosterone', 'applies', 'madhyaloka', 'kaludayin', 'circuits', 'adore', 'jorhat']


In [49]:
print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of sentences: {len(english_sentences)}")

Number of sentences: 124084
Number of sentences: 124084


In [50]:
def is_valid_tokens (sentence, vocab) :
    return all(token in vocab for token in sentence.split())
    
def is_valid_length (sentence, max_sequence_length) :
    return len(sentence.split()) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space
    
valid_sentence_indices = [
    index for index, (hindi_sentence, english_sentence) in enumerate(zip(hindi_sentences, english_sentences))
    if is_valid_length(hindi_sentence, max_sequence_length)
    and is_valid_length(english_sentence, max_sequence_length) 
    and is_valid_tokens(hindi_sentence, hindi_vocab)
]

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indices)}")

Number of sentences: 124084
Number of valid sentences: 124076


In [51]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indices]
english_sentences = [english_sentences[i] for i in valid_sentence_indices]
print(len(hindi_sentences))
print(len(english_sentences))

124076
124076


### Writing the valid processed files

In [52]:
file = open(train_en_file,'w')
for sentence in english_sentences:
    file.write(sentence+"\n")
file.close()

In [53]:
file = open(train_hn_file,'w')
for sentence in hindi_sentences:
    file.write(sentence+"\n")
file.close()

In [54]:
with open(train_en_file, 'r') as file:
    english_sentences = file.readlines()
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
english_sentences[:10]

['got it',
 'i m ok',
 'come in',
 'get out',
 'go away',
 'have fun',
 'have fun',
 'have fun',
 'i forgot',
 'i forgot']

In [55]:
with open(train_hn_file, 'r') as file:
    hindi_sentences = file.readlines()
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]
hindi_sentences[: 10]

['समझे कि नहीं',
 'मैं ठीक हूँ।',
 'अंदर आ जाओ।',
 'बाहर निकल जाओ',
 'चले जाओ',
 'मज़े करना।',
 'मौज करना।',
 'मज़े करो।',
 'मैं भूल गया।',
 'मैं भूल गई।']